In [15]:
import pandas as pd
import statsmodels.stats.multicomp as mc
from scipy.stats import anderson, shapiro
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import kruskal
from scikit_posthocs import posthoc_dunn
from scipy.stats import levene

# read RMSE, MAE, MAPE csv files
RMSE_df = pd.read_csv('/Users/musaphiri/Documents/Solar Tracking Research/Solar tracking dataset document/Solar tracking notebooks/csv_metrics/RMSE.csv')
MAE_df = pd.read_csv('/Users/musaphiri/Documents/Solar Tracking Research/Solar tracking dataset document/Solar tracking notebooks/csv_metrics/MAE.csv')
MAPE_df = pd.read_csv('/Users/musaphiri/Documents/Solar Tracking Research/Solar tracking dataset document/Solar tracking notebooks/csv_metrics/MAPE.csv')
#RMSE_df1 = pd.read_csv('/Users/musaphiri/Documents/Solar Tracking Research/Solar tracking dataset document/Solar tracking notebooks/RMSE.csv')

In [16]:
long_RMSE_df = pd.melt(RMSE_df, var_name='Group', value_name='Value')
long_MAE_df = pd.melt(MAE_df, var_name='Group', value_name='Value')
long_MAPE_df = pd.melt(MAPE_df, var_name='Group', value_name='Value')

# RMSE

In [17]:
# # Perform the Anderson-Darling test
# for group in RMSE_df.Group.unique():
#     statistic, critical_values, significance_level = anderson(RMSE_df[RMSE_df.Group==group].Value)
#     print(f"{group} Anderson-Darling statistic: {statistic}")
#     print(f"{group} Critical values: {critical_values}")
#     print(f"{group} Significance level: {significance_level}")

'''
The Shapiro-Wilk test returns a test statistic and 
a p-value. If the p-value is less than a certain significance 
level (usually 0.05), it is concluded that the data does not 
come from a normal distribution. If the p-value is greater than 
the significance level, it is concluded that there is not enough 
evidence to suggest that the data does not come from a normal distribution.
''' 

# Perform the Shapiro-Wilk test
for group in long_RMSE_df.Group.unique():
    statistic, p_value = shapiro(long_RMSE_df[long_RMSE_df.Group==group].Value)
    print(f"{group} Shapiro-Wilk statistic: {statistic}")
    print(f"{group} p-value: {p_value}")   

LSTM Shapiro-Wilk statistic: 0.8816819190979004
LSTM p-value: 0.0030820821411907673
CNN Shapiro-Wilk statistic: 0.9186841249465942
CNN p-value: 0.02479676343500614
SCT-CNN-LSTM Shapiro-Wilk statistic: 0.9402452707290649
SCT-CNN-LSTM p-value: 0.09235288202762604
GRU Shapiro-Wilk statistic: 0.9090702533721924
GRU p-value: 0.014099323190748692


In [18]:
# # create a pandas dataframe
# df = pd.DataFrame({'group': ['group1', 'group1', 'group1', 'group1', 'group1', 'group2', 'group2', 'group2', 'group2', 'group2','group3','group3','group3','group3','group3'],
#                   'value': [1.2, 2.3, 3.1, 4.5, 5.6, 2.3, 3.4, 4.1, 5.2, 6.3,1.1, 2.2,3.3,4.4,5.5]})

# perform the Brown-Forsythe test
stat, p_value = levene(*[long_RMSE_df[long_RMSE_df.Group==group].Value.values for group in long_RMSE_df.Group.unique()], center='median')

print("Statistic: ", stat)
print("P-value: ", p_value)

Statistic:  6.4858851750460635
P-value:  0.0004266312360187299


The Fligner-Killeen test and the Brown-Forsythe test are both non-parametric tests used to check if variances are equal across multiple groups. They are particularly useful when the data is not normally distributed and the sample sizes are not equal.

Fligner-Killeen test: This test is used to determine if the variances of two or more groups are equal. It compares the variances of each group to the overall variance of the data. It is a non-parametric test and it's based on the median of the data rather than the mean.

Brown-Forsythe test: This test is used to determine if the variances of two or more groups are equal. It compares the variances of each group to the overall variance of the data. It is a non-parametric test and it's based on the absolute deviation of the data from the median rather than the mean. The Brown-Forsythe test is a non-parametric test that can be used to check if variances are equal across multiple groups when the sample sizes are equal. It is based on the absolute deviation of the data from the median rather than the mean and it's robust to non-normality.

In [19]:
# Perform the Kruskal-Wallis test
statistic, p_value = kruskal(*[long_RMSE_df[long_RMSE_df.Group==group].Value.values for group in long_RMSE_df.Group.unique()])
print(f"Kruskal-Wallis statistic: {statistic}")
print(f"p-value: {p_value}")


if p_value < 0.05:
    print("The null hypothesis of equal medians can be rejected. There is a significant difference among the groups.")
else:
    print("The null hypothesis of equal medians cannot be rejected.")

Kruskal-Wallis statistic: 37.17145650014934
p-value: 4.232631860333199e-08
The null hypothesis of equal medians can be rejected. There is a significant difference among the groups.


In this example, the data is contained in a pandas dataframe, where the column "group" indicates the group to which each value belongs to. The test compares the variances of each group using the absolute deviation of the data from the median rather than the mean.

The test will return a p-value. If the p-value is less than the significance level (usually 0.05), the variances are considered not equal.

It's worth noting that this is a simple example, in a real-world scenario, you need to check the assumptions of your data, sample size and the significance level, and accordingly decide if the variances are equal or not.

In [20]:
# Perform the Dunn's test on the dataframe
dunn_results = posthoc_dunn(long_RMSE_df,'Value', 'Group')
dunn_results

,CNN,GRU,LSTM,SCT-CNN-LSTM
CNN,1.000000,2.817955e-01,0.088819,3.658020e-06
GRU,0.281795,1.000000e+00,0.005470,1.155144e-08
LSTM,0.088819,5.470241e-03,1.000000,3.408607e-03
SCT-CNN-LSTM,0.000004,1.155144e-08,0.003409,1.000000e+00


The results of a Dunn's test are typically reported as a matrix of p-values. Each element in the matrix corresponds to a pair of groups, and the value represents the p-value for the null hypothesis that the means of the two groups are equal.

A small p-value (typically less than 0.05) indicates that there is strong evidence to reject the null hypothesis, and that there is a significant difference in means between the two groups. A large p-value (typically greater than 0.05) indicates that there is not enough evidence to reject the null hypothesis, and that there is not a significant difference in means between the two groups.

It is important to adjust the p-value for multiple comparisons, this is done by applying a correction method such as bonferroni or holm correction. This helps to control the family wise error rate.

When interpreting the results, it's also important to consider the context of the experiment and any external information that might be relevant. For example, if the groups being compared are known to have distinct characteristics or if the difference in means is large enough, it might be reasonable to conclude that the groups are significantly different even if the p-value is not quite small enough to reject the null hypothesis.

# MAE

In [21]:
'''
The Shapiro-Wilk test returns a test statistic and 
a p-value. If the p-value is less than a certain significance 
level (usually 0.05), it is concluded that the data does not 
come from a normal distribution. If the p-value is greater than 
the significance level, it is concluded that there is not enough 
evidence to suggest that the data does not come from a normal distribution.
''' 

# Perform the Shapiro-Wilk test
for group in long_MAE_df.Group.unique():
    statistic, p_value = shapiro(long_MAE_df[long_MAE_df.Group==group].Value)
    print(f"{group} Shapiro-Wilk statistic: {statistic}")
    print(f"{group} p-value: {p_value}")   

LSTM Shapiro-Wilk statistic: 0.8857030868530273
LSTM p-value: 0.0038217699620872736
CNN Shapiro-Wilk statistic: 0.8638902902603149
CNN p-value: 0.001228554523549974
SCT-CNN-LSTM Shapiro-Wilk statistic: 0.9547849893569946
SCT-CNN-LSTM p-value: 0.22666142880916595
GRU Shapiro-Wilk statistic: 0.9141759872436523
GRU p-value: 0.018992271274328232


In [22]:
# perform the Brown-Forsythe test
stat, p_value = levene(*[long_MAE_df[long_MAE_df.Group==group].Value.values for group in long_MAE_df.Group.unique()], center='median')

print("Statistic: ", stat)
print("P-value: ", p_value)

Statistic:  5.122547555340878
P-value:  0.002305570685708242


In [23]:
# Perform the Kruskal-Wallis test
statistic, p_value = kruskal(*[long_MAE_df[long_MAE_df.Group==group].Value.values for group in long_MAE_df.Group.unique()])
print(f"Kruskal-Wallis statistic: {statistic}")
print(f"p-value: {p_value}")


if p_value < 0.05:
    print("The null hypothesis of equal medians can be rejected. There is a significant difference among the groups.")
else:
    print("The null hypothesis of equal medians cannot be rejected.")

Kruskal-Wallis statistic: 33.35437523780362
p-value: 2.7113515861046045e-07
The null hypothesis of equal medians can be rejected. There is a significant difference among the groups.


In [24]:
# Perform the Dunn's test on the dataframe
dunn_results = posthoc_dunn(long_MAE_df,'Value', 'Group')
dunn_results

,CNN,GRU,LSTM,SCT-CNN-LSTM
CNN,1.000000,7.392002e-02,0.454066,1.123890e-04
GRU,0.073920,1.000000e+00,0.011220,1.611326e-08
LSTM,0.454066,1.122031e-02,1.000000,1.848785e-03
SCT-CNN-LSTM,0.000112,1.611326e-08,0.001849,1.000000e+00


# MAPE

In [25]:
'''
The Shapiro-Wilk test returns a test statistic and 
a p-value. If the p-value is less than a certain significance 
level (usually 0.05), it is concluded that the data does not 
come from a normal distribution. If the p-value is greater than 
the significance level, it is concluded that there is not enough 
evidence to suggest that the data does not come from a normal distribution.
''' 

# Perform the Shapiro-Wilk test
for group in long_MAPE_df.Group.unique():
    statistic, p_value = shapiro(long_MAPE_df[long_MAPE_df.Group==group].Value)
    print(f"{group} Shapiro-Wilk statistic: {statistic}")
    print(f"{group} p-value: {p_value}")   

LSTM Shapiro-Wilk statistic: 0.8850650191307068
LSTM p-value: 0.0036928465124219656
CNN Shapiro-Wilk statistic: 0.9300492405891418
CNN p-value: 0.049251098185777664
SCT-CNN-LSTM Shapiro-Wilk statistic: 0.9725630283355713
SCT-CNN-LSTM p-value: 0.6114177703857422
GRU Shapiro-Wilk statistic: 0.8401433825492859
GRU p-value: 0.0003881706506945193


In [26]:
# perform the Brown-Forsythe test
stat, p_value = levene(*[long_MAPE_df[long_MAPE_df.Group==group].Value.values for group in long_MAPE_df.Group.unique()], center='median')

print("Statistic: ", stat)
print("P-value: ", p_value)

Statistic:  4.290548508501065
P-value:  0.0065488444023000456


In [27]:
# Perform the Kruskal-Wallis test
statistic, p_value = kruskal(*[long_MAPE_df[long_MAPE_df.Group==group].Value.values for group in long_MAPE_df.Group.unique()])
print(f"Kruskal-Wallis statistic: {statistic}")
print(f"p-value: {p_value}")


if p_value < 0.05:
    print("The null hypothesis of equal medians can be rejected. There is a significant difference among the groups.")
else:
    print("The null hypothesis of equal medians cannot be rejected.")

Kruskal-Wallis statistic: 17.909009084090332
p-value: 0.0004592699940952509
The null hypothesis of equal medians can be rejected. There is a significant difference among the groups.


In [28]:
# Perform the Dunn's test on the dataframe
dunn_results = posthoc_dunn(long_MAPE_df,'Value', 'Group')
dunn_results

,CNN,GRU,LSTM,SCT-CNN-LSTM
CNN,1.000000,0.823783,0.738362,0.000284
GRU,0.823783,1.000000,0.911346,0.000657
LSTM,0.738362,0.911346,1.000000,0.000982
SCT-CNN-LSTM,0.000284,0.000657,0.000982,1.000000
